In [2]:
!pip install ultralytics
!pip install supervision
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.1/800.1 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
#**********************************LIBRARIES*********************************#
from ultralytics import YOLO
import supervision as sv
import pickle
import os
import cv2

# INPUT-video file
video_path = 'video_input.mp4'
# OUTPUT-Video File
output_video_path = 'output_video.mp4'
# PICKLE FILE (IF AVAILABLE LOADS IT IF NOT, SAVES IT IN THIS PATH)
pickle_path = 'track_stubs.pkl'

In [4]:
#*********************************TRACKING MECHANISM**************************#
class HockeyAnalyzer:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()

    def detect_frames(self, frames):
        batch_size = 20
        detections = []
        for i in range(0, len(frames), batch_size):
            detections_batch = self.model.predict(frames[i:i+batch_size], conf=0.1)
            detections += detections_batch
        return detections

#********LOAD TRACKS FROM FILE OR DETECT OBJECTS-SAVES PICKLE FILE************#

    def get_object_tracks(self, frames, read_from_stub=False, stub_path=None):
        if read_from_stub and stub_path is not None and os.path.exists(stub_path):
            with open(stub_path, 'rb') as f:
                tracks = pickle.load(f)
            return tracks

        detections = self.detect_frames(frames)

        tracks = {"person": []}

        for frame_num, detection in enumerate(detections):
            cls_names = detection.names
            cls_names_inv = {v: k for k, v in cls_names.items()}

            # Tracking Mechanism
            detection_supervision = sv.Detections.from_ultralytics(detection)
            detection_with_tracks = self.tracker.update_with_detections(detection_supervision)
            tracks["person"].append({})

            for frame_detection in detection_with_tracks:
                bbox = frame_detection[0].tolist()
                cls_id = frame_detection[3]
                track_id = frame_detection[4]

                if cls_id == cls_names_inv.get('person', None):
                    tracks["person"][frame_num][track_id] = {"bbox": bbox}

            for frame_detection in detection_supervision:
                bbox = frame_detection[0].tolist()
                cls_id = frame_detection[3]

        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(tracks, f)

        return tracks

#***********************BOUNDING BOXES AND TRACK-IDs**************************#

    def draw_annotations(self, video_frames, tracks):
        output_video_frames = []
        for frame_num, frame in enumerate(video_frames):
            frame = frame.copy()
            player_dict = tracks["person"][frame_num]

            # Draw Players
            for track_id, player in player_dict.items():
                color = player.get("team_color", (0, 0, 255))
                bbox = player["bbox"]
                x1, y1, x2, y2 = map(int, bbox)
            # Bounding boxes
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Track_id
                cv2.putText(frame, str(track_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            output_video_frames.append(frame)

        return output_video_frames

In [6]:
#*************** EXECUTES TRACKING MECHANISM AND OUTPUT VIDEO****************#

# Read the video frames
video_frames = []
cap = cv2.VideoCapture(video_path)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    video_frames.append(frame)
cap.release()

#********************* EXECUTE TRACKING METHOD WITH YOLO**********************#
tracker = HockeyAnalyzer('yolov8x.pt')
tracks = tracker.get_object_tracks(video_frames, read_from_stub=True, stub_path=pickle_path)
annotated_frames = tracker.draw_annotations(video_frames, tracks)

#*********************** SAVES VIDEO FILE ************************************#
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height, width, _ = annotated_frames[0].shape
out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

for frame in annotated_frames:
    out.write(frame)
out.release()

100%|██████████| 131M/131M [00:00<00:00, 194MB/s]



0: 384x640 28 persons, 3233.6ms
1: 384x640 26 persons, 3233.6ms
2: 384x640 28 persons, 3233.6ms
3: 384x640 28 persons, 3233.6ms
4: 384x640 28 persons, 3233.6ms
5: 384x640 31 persons, 3233.6ms
6: 384x640 27 persons, 3233.6ms
7: 384x640 27 persons, 3233.6ms
8: 384x640 26 persons, 3233.6ms
9: 384x640 28 persons, 3233.6ms
10: 384x640 26 persons, 3233.6ms
11: 384x640 29 persons, 3233.6ms
12: 384x640 27 persons, 3233.6ms
13: 384x640 28 persons, 3233.6ms
14: 384x640 28 persons, 3233.6ms
15: 384x640 31 persons, 2 skiss, 3233.6ms
16: 384x640 29 persons, 1 skis, 3233.6ms
17: 384x640 28 persons, 1 skis, 3233.6ms
18: 384x640 25 persons, 1 skis, 3233.6ms
19: 384x640 24 persons, 1 skis, 3233.6ms
Speed: 9.1ms preprocess, 3233.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 persons, 1 skis, 3155.5ms
1: 384x640 26 persons, 2 skiss, 3155.5ms
2: 384x640 25 persons, 2 skiss, 3155.5ms
3: 384x640 26 persons, 2 skiss, 3155.5ms
4: 384x640 29 persons, 2 skiss, 3155.5ms
5: 3

In [8]:
#************ Design of Ellipse for tracking players instead of Bounding boxes**************#

def draw_ellipse(self, frame, bbox, color, track_id=None, team=None):
  y2 = int(bbox[3])
  x_center = (int(bbox[0]) + int(bbox[2])) // 2
  width = int(bbox[2]) - int(bbox[0])
  color = (255, 0, 0)
  text_color = (255, 255, 255)

  cv2.ellipse(
      frame,
      center=(x_center, y2),
      axes=(int(width) // 2, int(0.35 * width)),
      angle=0.0,
      startAngle=-45,
      endAngle=235,
      color=color,
      thickness=2,
      lineType=cv2.LINE_4
  )

  if track_id is not None:
      rectangle_width = 40
      rectangle_height = 20
      x1_rect = x_center - rectangle_width // 2
      x2_rect = x_center + rectangle_width // 2
      y1_rect = (y2 - rectangle_height // 2) + 15
      y2_rect = (y2 + rectangle_height // 2) + 15

      cv2.rectangle(frame,
                    (int(x1_rect), int(y1_rect)),
                    (int(x2_rect), int(y2_rect)),
                    color,
                    cv2.FILLED)

      x1_text = x1_rect + 12
      if track_id > 99:
          x1_text -= 10
      font_scale = 0.4
      cv2.putText(
          frame,
          f"{track_id}",
          (int(x1_text), int(y1_rect + 15)),
          cv2.FONT_HERSHEY_SIMPLEX,
          font_scale,
          text_color,
          thickness=2
      )

  return frame

In [9]:
#***********************BOUNDING BOXES AND TRACK-IDs**************************#

def draw_annotations(self, video_frames, tracks):
  output_video_frames = []
  for frame_num, frame in enumerate(video_frames):
      frame = frame.copy()
      player_dict = tracks["person"][frame_num]

      # Draw Players
      for track_id, player in player_dict.items():
          bbox = player["bbox"]

      # Draw ellipse and tracking IDs
          self.draw_ellipse(frame, bbox, (0, 255, 0), track_id)

          x1, y1, x2, y2 = map(int, bbox)

      output_video_frames.append(frame)

  return output_video_frames


In [11]:
#********************* Border Definition for Frame***********************
import cv2

video_path = 'video_input.mp4'
cap = cv2.VideoCapture(video_path)

#**************Read, Define and Draw corners of the frame****************
ret, frame = cap.read()

bottom_left = (0, 720)
bottom_right = (1280, 720)
upper_left = (0, 0)
upper_right = (1280, 0)

cv2.line(frame, bottom_left, bottom_right, (0, 255, 0), 2)
cv2.line(frame, bottom_left, upper_left, (0, 255, 0), 2)
cv2.line(frame, bottom_right, upper_right, (0, 255, 0), 2)
cv2.line(frame, upper_left, upper_right, (0, 255, 0), 2)

#*******************Save the frame with marked corners*********************
output_image_path = 'rink_area_marked_VALIDATION.png'
cv2.imwrite(output_image_path, frame)
print("Rink area saved:", output_image_path)

Rink area saved: rink_area_marked_VALIDATION.png


**************YELLOW TEAM OFFENSIVE ZONE****************

Bottom Left Corner: (-450, 710)

Bottom Right Corner: (2030, 710)

Upper Left Corner: (200, 150)

Upper Right Corner: (1160, 150)

**************WHITE TEAM OFFENSIVE ZONE****************

Bottom Left Corner: (180, 150)

Bottom Right Corner: (1100, 150)

Upper Left Corner: (352, 61)

Upper Right Corner: (900, 61)

In [12]:
!pip install torch torchvision
!pip install matplotlib
!pip install scikit-learn

In [1]:
# ************CONVOLUTIONAL NEURAL NETWORK-THREE CLASSES DETECTION**************************
# REFEREE
# WHITE TEAM (Team_away)
# YELLOW TEAM (Team_home)

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

#Training and Validation Datasets
#Download the teams_sample_dataset file from the project's GitHub repository
data_dir = 'teams_sample_dataset'

In [16]:
#******************************Data transformation***********************************
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
#********************************CNN Model Architecture**************************************
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 3)  #Three Classes (Referee, Team_away,Team_home)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
#********************************CNN TRAINING**********************************************

# Model-loss function-optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#*********************************Training*************************************************
num_epochs = 10
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.type(torch.LongTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = labels.type(torch.LongTensor)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.tolist())
            all_preds.extend(preds.tolist())

In [ ]:
#********************************METRICS & PERFORMANCE************************************

    val_losses.append(val_loss / len(val_loader))
    val_accuracy = accuracy_score(all_labels, all_preds)
    val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=1)
    val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)

    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {train_losses[-1]:.4f}, "
          f"Val Loss: {val_losses[-1]:.4f}, "
          f"Val Acc: {val_accuracy:.2%}, "
          f"Val Precision: {val_precision:.4f}, "
          f"Val Recall: {val_recall:.4f}, "
          f"Val F1 Score: {val_f1:.4f}")

#*******************************SHOW METRICS & PERFORMANCE**********************************
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.show()

# SAVE THE MODEL FOR THE GH_CV_track_teams CODE
torch.save(model.state_dict(), 'hockey_team_classifier.pth')

In [ ]:
# *************TEST CNN MODEL WITH SAMPLE DATASET***************************

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

# SAMPLE DATASET FOR VALIDATION
test_dir = 'D:/PYTHON/validation_dataset'

# CNN MODEL FOR TEAM PREDICTIONS
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# CNN MODEL PREVIOUSLY SAVED
model = CNNModel()
model.load_state_dict(torch.load('D:/PYTHON/hockey_team_classifier.pth'))
model.eval()

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

#******************ITERATION ON SAMPLE IMAGES-ACCURACY TEST*****************************

class_names = ['team_referee', 'team_away', 'team_home']

def predict_image(image_path, model, transform):
# LOADS DATASET
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

# MAKES PREDICTIONS
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        team = class_names[predicted.item()]
    return team

for image_name in os.listdir(test_dir):
    image_path = os.path.join(test_dir, image_name)
    if os.path.isfile(image_path):
        predicted_team = predict_image(image_path, model, transform)
        print(f'Image {image_name}: The player belongs to {predicted_team}')